In [17]:
# Environment Setup and Library Imports
import sys
import os
from pathlib import Path
import importlib

# Configure project root for module imports
project_root = Path(os.getcwd()).parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Reload modules for latest functions
import src.analysis
importlib.reload(src.analysis)

from src.loader import load_data, clean_data
from src.analysis import (calculate_driver_stats, calculate_team_stats,
                          calculate_combined_standings, calculate_combined_constructor_standings)
from src.features import prepare_features
from src.model import train_model, load_trained_model
from src.evaluate import evaluate_model

plt.style.use('seaborn-v0_8-darkgrid')
print('Environment configured successfully.')

Environment configured successfully.


## 1. Load Season Data (Race + Sprint)
Load both race and sprint results for accurate championship standings calculation.

In [ ]:
# Load 2025 Season Race and Sprint Results
race_path = project_root / 'data' / 'Formula1_2025Season_RaceResults.csv'
sprint_path = project_root / 'data' / 'Formula1_2025Season_SprintResults.csv'

df_race = load_data(str(race_path))
df_sprint = load_data(str(sprint_path))

df_race = clean_data(df_race)

# Fix sprint data types
df_sprint['Points'] = pd.to_numeric(df_sprint['Points'], errors='coerce').fillna(0)
df_sprint['Position'] = pd.to_numeric(df_sprint['Position'], errors='coerce')

print(f'Race Data: {len(df_race)} entries from {df_race["Track"].nunique()} Grand Prix')
print(f'Sprint Data: {len(df_sprint)} entries from {df_sprint["Track"].nunique()} Sprint events')
print(f'Drivers: {df_race["Driver"].nunique()} | Teams: {df_race["Team"].nunique()}')
print(f'\n2025 Grand Prix Calendar:')
print(list(df_race['Track'].unique()))

Loaded 459 race entries from 23 races
Drivers: 21
Teams: 12

Races in season:
['Australia' 'China' 'Japan' 'Bahrain' 'Saudi Arabia' 'Miami'
 'Emilia-Romagna' 'Monaco' 'Spain' 'Canada' 'Austria' 'Great Britain'
 'Belgium' 'Hungary' 'Netherlands' 'Italy' 'Azerbaijan' 'Singapore'
 'United States' 'Mexico' 'Brazil' 'Las Vegas' 'Qatar']


## 2. Current Championship Standings (Race + Sprint)
Official FIA standings combining both race and sprint points as per regulations.

In [ ]:
# Calculate Combined Championship Standings (Race + Sprint)
driver_stats = calculate_driver_stats(df_race)
team_stats = calculate_team_stats(df_race)

# Official combined standings
combined_driver_standings = calculate_combined_standings(df_race, df_sprint)
combined_team_standings = calculate_combined_constructor_standings(df_race, df_sprint)

print('=' * 70)
print("         2025 FIA FORMULA 1 DRIVERS' WORLD CHAMPIONSHIP")
print('         (After Qatar GP - Race + Sprint Points)')
print('=' * 70)

standings_display = combined_driver_standings.head(10).copy()
standings_display = standings_display.reset_index()
standings_display.insert(0, 'Pos', range(1, len(standings_display) + 1))
standings_display[['Pos', 'Driver', 'Team', 'Race_Points', 'Sprint_Points', 'Total_Points', 'Race_Wins']]

=== DRIVER CHAMPIONSHIP STANDINGS ===


,Position,Total_Points,Wins,Podium,Avg_Position
Driver,,,,,
Lando Norris,1,379,7.0,17.0,3.64
Max Verstappen,2,364,7.0,14.0,3.14
Oscar Piastri,3,363,7.0,15.0,2.81
George Russell,4,279,2.0,9.0,4.52
Charles Leclerc,5,213,0.0,7.0,5.10
Kimi Antonelli,6,135,0.0,3.0,7.84
Lewis Hamilton,7,131,0.0,0.0,6.95
Alexander Albon,8,70,0.0,0.0,9.32
Carlos Sainz,9,54,0.0,2.0,10.63


In [ ]:
# Constructor Championship Standings (Race + Sprint)
print('=' * 70)
print("         2025 FIA FORMULA 1 CONSTRUCTORS' WORLD CHAMPIONSHIP")
print('         (After Qatar GP - Race + Sprint Points)')
print('=' * 70)

constructor_display = combined_team_standings.reset_index()
constructor_display.insert(0, 'Pos', range(1, len(constructor_display) + 1))
constructor_display[['Pos', 'Team', 'Race_Points', 'Sprint_Points', 'Total_Points']]

=== CONSTRUCTOR CHAMPIONSHIP STANDINGS ===


,Total_Points,Avg_Position,Finish_Rate
Team,,,
McLaren Mercedes,732,3.19,91.3
Mercedes,414,6.02,91.3
Red Bull Racing Honda RBPT,385,7.36,60.9
Ferrari,344,6.02,87.0
Williams Mercedes,124,9.97,82.6
Racing Bulls Honda RBPT,88,11.10,58.0
Aston Martin Aramco Mercedes,71,12.21,84.8
Kick Sauber Ferrari,68,12.46,80.4
Haas Ferrari,67,11.57,91.3


## 3. Model Training and Accuracy Metrics
Train Random Forest model with detailed accuracy and confidence metrics.

In [ ]:
# Train Random Forest Model with Cross-Validation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print('Training Random Forest model on 2025 season data...\n')
model, X_test, y_test = train_model(df_race)

# Evaluate model with detailed metrics
metrics, results = evaluate_model(model, X_test, y_test)

# Cross-validation for confidence estimation
X_full, y_full = prepare_features(df_race)
cv_scores = cross_val_score(model, X_full, y_full, cv=5, scoring='neg_mean_absolute_error')
cv_mae = -cv_scores.mean()
cv_std = cv_scores.std()

print('=' * 60)
print('              MODEL PERFORMANCE METRICS')
print('=' * 60)
print(f'''
  Mean Absolute Error (MAE):     {metrics["MAE"]:.2f} positions
  Root Mean Squared Error:       {metrics["RMSE"]:.2f} positions
  R² Score:                      {metrics["R2"]:.3f}
  
  Cross-Validation MAE:          {cv_mae:.2f} ± {cv_std:.2f} positions
  Model Accuracy:                {(1 - cv_mae/10) * 100:.1f}%
  
  Prediction Confidence:         ±{cv_mae + cv_std:.1f} positions (95% CI)
''')
print('=' * 60)

Training model on 2025 season data...
Preparing features...
Training model...
Model saved to models/f1_model.pkl

Model Performance:
  MAE: 2.55 positions
  R2 Score: 0.523


In [ ]:
# Feature Importance Analysis
feature_names = ['Starting Grid', 'Driver', 'Team', 'Track']
importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=True)

fig = px.bar(importance, x='Importance', y='Feature', orientation='h',
             title='Feature Importance for Position Prediction',
             color='Importance', color_continuous_scale='Reds')
fig.update_layout(template='plotly_dark')
fig.show()

# Prediction vs Actual scatter
fig = px.scatter(results, x='Actual', y='Predicted',
                 title='Model Validation: Predicted vs Actual Positions',
                 labels={'Actual': 'Actual Position', 'Predicted': 'Predicted Position'})
fig.add_shape(type='line', x0=1, y0=1, x1=20, y1=20, line=dict(dash='dash', color='red'))
fig.update_layout(template='plotly_dark')
fig.show()

## 4. Abu Dhabi GP 2025 - Grid Prediction
Estimated starting grid based on season qualifying performance metrics.

In [ ]:
# Generate Predicted Starting Grid based on Season Qualifying Performance
drivers_teams = df_race.groupby('Driver')['Team'].first().reset_index()

# Calculate average grid position (proxy for qualifying pace)
avg_grid = df_race.groupby('Driver')['Starting Grid'].mean().reset_index()
avg_grid.columns = ['Driver', 'Avg_Grid']

# Calculate grid consistency (standard deviation)
grid_std = df_race.groupby('Driver')['Starting Grid'].std().reset_index()
grid_std.columns = ['Driver', 'Grid_Consistency']

# Merge data
abu_dhabi_grid = drivers_teams.merge(avg_grid, on='Driver').merge(grid_std, on='Driver')
abu_dhabi_grid = abu_dhabi_grid.sort_values('Avg_Grid').reset_index(drop=True)
abu_dhabi_grid['Predicted_Grid'] = range(1, len(abu_dhabi_grid) + 1)

print('=' * 70)
print('         ABU DHABI GP 2025 - PREDICTED STARTING GRID')
print('         Yas Marina Circuit - December 7, 2025')
print('=' * 70)
print('(Based on average qualifying performance this season)\n')
abu_dhabi_grid[['Predicted_Grid', 'Driver', 'Team', 'Avg_Grid', 'Grid_Consistency']].head(20)

=== PREDICTED ABU DHABI STARTING GRID ===
(Based on average qualifying performance this season)


,Predicted_Grid,Driver,Team,Avg_Grid
0,1,Oscar Piastri,McLaren Mercedes,3.000000
1,2,Lando Norris,McLaren Mercedes,3.000000
2,3,Max Verstappen,Red Bull Racing Honda RBPT,3.695652
3,4,George Russell,Mercedes,4.391304
4,5,Charles Leclerc,Ferrari,5.434783
5,6,Kimi Antonelli,Mercedes,8.521739
6,7,Lewis Hamilton,Ferrari,9.086957
7,8,Isack Hadjar,Racing Bulls Honda RBPT,9.826087
8,9,Fernando Alonso,Aston Martin Aramco Mercedes,10.391304
9,10,Carlos Sainz,Williams Mercedes,11.130435


In [ ]:
# Prepare Prediction Data with Encoders
from sklearn.preprocessing import LabelEncoder
import pickle

model_dir = project_root / 'models'
encoders = {}
for col in ['Driver', 'Team', 'Track']:
    encoder_path = model_dir / f'{col}_encoder.pkl'
    with open(encoder_path, 'rb') as f:
        encoders[col] = pickle.load(f)

# Create prediction dataframe
abu_dhabi_pred = abu_dhabi_grid.copy()
abu_dhabi_pred['Track'] = 'Abu Dhabi'
abu_dhabi_pred['Starting Grid'] = abu_dhabi_pred['Predicted_Grid']

# Handle track encoding
if 'Abu Dhabi' not in encoders['Track'].classes_:
    print('Note: Abu Dhabi not in training data, using proxy track encoding')
    available_tracks = list(encoders['Track'].classes_)
    abu_dhabi_pred['Track'] = available_tracks[0]

print('Prediction data prepared successfully.')

Note: Abu Dhabi not in training data, using similar track encoding
Prediction data prepared


c:\Users\test\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning:

Trying to unpickle estimator LabelEncoder from version 1.7.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



In [ ]:
# Generate Race Predictions with Confidence Intervals
X_pred = abu_dhabi_pred[['Starting Grid', 'Driver', 'Team', 'Track']].copy()

for col in ['Driver', 'Team', 'Track']:
    le = encoders[col]
    X_pred[col] = X_pred[col].apply(
        lambda x: le.transform([x])[0] if x in le.classes_ else -1
    )

# Ensemble prediction from all trees for confidence interval
all_predictions = np.array([tree.predict(X_pred) for tree in model.estimators_])
predicted_positions = all_predictions.mean(axis=0)
prediction_std = all_predictions.std(axis=0)

# Add to dataframe
abu_dhabi_pred['Predicted_Position'] = predicted_positions
abu_dhabi_pred['Prediction_Std'] = prediction_std
abu_dhabi_pred['Confidence_Lower'] = np.maximum(1, predicted_positions - 1.96 * prediction_std)
abu_dhabi_pred['Confidence_Upper'] = np.minimum(20, predicted_positions + 1.96 * prediction_std)

# Sort by predicted position
final_prediction = abu_dhabi_pred.sort_values('Predicted_Position').reset_index(drop=True)
final_prediction['Final_Position'] = range(1, len(final_prediction) + 1)

print('Predictions generated with 95% confidence intervals.')

## 5. Abu Dhabi GP 2025 - Predicted Race Results
Race classification prediction with confidence intervals and points allocation.

In [ ]:
# Calculate Predicted Points with Confidence Ranges
points_system = {1: 25, 2: 18, 3: 15, 4: 12, 5: 10, 6: 8, 7: 6, 8: 4, 9: 2, 10: 1}

final_prediction['Predicted_Points'] = final_prediction['Final_Position'].map(
    lambda x: points_system.get(x, 0)
)

# Calculate confidence percentage (based on prediction std)
final_prediction['Confidence_%'] = (100 - final_prediction['Prediction_Std'] * 10).clip(50, 99).round(1)

print('=' * 80)
print('                 ABU DHABI GP 2025 - PREDICTED RACE RESULTS')
print('                 Yas Marina Circuit - December 7, 2025')
print('=' * 80)

result_display = final_prediction[[
    'Final_Position', 'Driver', 'Team', 'Predicted_Grid', 
    'Predicted_Points', 'Confidence_%'
]].copy()
result_display.columns = ['Pos', 'Driver', 'Team', 'Grid', 'Points', 'Confidence']
result_display['Confidence'] = result_display['Confidence'].apply(lambda x: f'{x}%')
result_display

       ABU DHABI GP 2025 - PREDICTED RACE RESULTS
       Yas Marina Circuit - December 7, 2025


,Pos,Driver,Team,Grid,Points
0,1,Max Verstappen,Red Bull Racing Honda RBPT,3,25
1,2,Oscar Piastri,McLaren Mercedes,1,18
2,3,George Russell,Mercedes,4,15
3,4,Lando Norris,McLaren Mercedes,2,12
4,5,Charles Leclerc,Ferrari,5,10
5,6,Lewis Hamilton,Ferrari,7,8
6,7,Kimi Antonelli,Mercedes,6,6
7,8,Liam Lawson,Red Bull Racing Honda RBPT,11,4
8,9,Nico Hulkenberg,Kick Sauber Ferrari,21,2
9,10,Fernando Alonso,Aston Martin Aramco Mercedes,9,1


In [ ]:
# Visualize Predicted Top 10 with Confidence Intervals
top_10 = final_prediction.head(10)
colors = ['gold', 'silver', '#CD7F32'] + ['#E10600'] * 7

fig = go.Figure()

fig.add_trace(go.Bar(
    x=top_10['Driver'],
    y=top_10['Predicted_Points'],
    marker_color=colors,
    text=top_10.apply(lambda x: f"P{x['Final_Position']}<br>{x['Confidence_%']}%", axis=1),
    textposition='outside',
    error_y=dict(
        type='data',
        symmetric=False,
        array=top_10['Prediction_Std'] * 2,
        arrayminus=top_10['Prediction_Std'] * 2,
        visible=True
    )
))

fig.update_layout(
    title='Abu Dhabi GP 2025 - Predicted Top 10 with Confidence Intervals',
    xaxis_title='Driver',
    yaxis_title='Predicted Points',
    template='plotly_dark',
    showlegend=False
)
fig.show()

In [ ]:
# Grid vs Predicted Finish Position with Confidence Bands
fig = go.Figure()

# Add confidence band
fig.add_trace(go.Scatter(
    x=list(final_prediction['Predicted_Grid']) + list(final_prediction['Predicted_Grid'][::-1]),
    y=list(final_prediction['Confidence_Lower']) + list(final_prediction['Confidence_Upper'][::-1]),
    fill='toself',
    fillcolor='rgba(225,6,0,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='95% Confidence'
))

# Add predicted positions
fig.add_trace(go.Scatter(
    x=final_prediction['Predicted_Grid'],
    y=final_prediction['Final_Position'],
    mode='markers+text',
    text=final_prediction['Driver'].apply(lambda x: x.split()[-1][:3].upper()),
    textposition='top center',
    marker=dict(size=12, color='#E10600'),
    name='Predicted'
))

# Add diagonal line
fig.add_trace(go.Scatter(
    x=[1, 20], y=[1, 20],
    mode='lines',
    line=dict(dash='dash', color='gray'),
    name='No change'
))

fig.update_layout(
    title='Grid Position vs Predicted Finish (with 95% Confidence Band)',
    xaxis_title='Grid Position',
    yaxis_title='Predicted Finish Position',
    template='plotly_dark',
    xaxis=dict(range=[0, 21], dtick=2),
    yaxis=dict(range=[0, 21], dtick=2, autorange='reversed')
)
fig.show()

## 6. Championship Impact Analysis
Predicted final championship standings after Abu Dhabi GP (Race + Sprint combined).

In [ ]:
# Calculate Predicted Final Championship Standings (Race + Sprint + Abu Dhabi Prediction)
current_standings = combined_driver_standings[['Total_Points', 'Race_Points', 'Sprint_Points']].copy()
current_standings = current_standings.reset_index()

# Merge with Abu Dhabi predicted points
abu_dhabi_points = final_prediction[['Driver', 'Predicted_Points', 'Confidence_%']]
final_standings = current_standings.merge(abu_dhabi_points, on='Driver', how='left')
final_standings['Predicted_Points'] = final_standings['Predicted_Points'].fillna(0)
final_standings['Confidence_%'] = final_standings['Confidence_%'].fillna(0)

# Calculate final totals
final_standings['Final_Total'] = final_standings['Total_Points'] + final_standings['Predicted_Points']
final_standings = final_standings.sort_values('Final_Total', ascending=False).reset_index(drop=True)
final_standings['Final_Position'] = range(1, len(final_standings) + 1)

# Calculate position change from current standings
current_pos = {d: i+1 for i, d in enumerate(current_standings.sort_values('Total_Points', ascending=False)['Driver'])}
final_standings['Position_Change'] = final_standings['Driver'].map(current_pos) - final_standings['Final_Position']

print('=' * 85)
print('         PREDICTED FINAL 2025 CHAMPIONSHIP STANDINGS')
print('         (After Abu Dhabi GP - Race + Sprint + Predicted Abu Dhabi Points)')
print('=' * 85)

display_standings = final_standings[[
    'Final_Position', 'Driver', 'Total_Points', 'Predicted_Points', 
    'Final_Total', 'Position_Change', 'Confidence_%'
]].head(15).copy()
display_standings.columns = ['Pos', 'Driver', 'Current Pts', 'Abu Dhabi', 'Final Total', 'Change', 'Conf %']
display_standings['Change'] = display_standings['Change'].apply(lambda x: f'+{int(x)}' if x > 0 else (f'{int(x)}' if x < 0 else '-'))
display_standings

=== PREDICTED FINAL CHAMPIONSHIP STANDINGS ===
(After Abu Dhabi GP)


,Final_Position,Driver,Total_Points,Predicted_Points,Final_Total
0,1,Lando Norris,379,12,391
1,2,Max Verstappen,364,25,389
2,3,Oscar Piastri,363,18,381
3,4,George Russell,279,15,294
4,5,Charles Leclerc,213,10,223
5,6,Kimi Antonelli,135,6,141
6,7,Lewis Hamilton,131,8,139
7,8,Alexander Albon,70,0,70
8,9,Carlos Sainz,54,0,54
9,10,Nico Hulkenberg,49,2,51


In [ ]:
# Visualize Championship Impact - Before vs After Abu Dhabi
top_8_standings = final_standings.head(8)

fig = go.Figure()

fig.add_trace(go.Bar(
    name='Race Points',
    x=top_8_standings['Driver'],
    y=top_8_standings['Race_Points'],
    marker_color='#1E1E1E',
    text=top_8_standings['Race_Points'].astype(int),
    textposition='inside'
))

fig.add_trace(go.Bar(
    name='Sprint Points',
    x=top_8_standings['Driver'],
    y=top_8_standings['Sprint_Points'],
    marker_color='#FF8000',
    text=top_8_standings['Sprint_Points'].astype(int),
    textposition='inside'
))

fig.add_trace(go.Bar(
    name='Abu Dhabi (Predicted)',
    x=top_8_standings['Driver'],
    y=top_8_standings['Predicted_Points'],
    marker_color='#E10600',
    text=top_8_standings['Predicted_Points'].astype(int),
    textposition='inside'
))

fig.update_layout(
    title='Predicted Final Championship - Points Breakdown',
    barmode='stack',
    template='plotly_dark',
    yaxis_title='Total Points',
    legend=dict(orientation='h', yanchor='bottom', y=1.02)
)
fig.show()

## 7. Prediction Summary and Accuracy Report
Comprehensive summary of predictions with confidence metrics and accuracy analysis.

In [ ]:
winner = final_prediction.iloc[0]
p2 = final_prediction.iloc[1]
p3 = final_prediction.iloc[2]
champion = final_standings.iloc[0]

print('=' * 70)
print('             ABU DHABI GP 2025 - PREDICTION SUMMARY')
print('             Yas Marina Circuit - December 7, 2025')
print('=' * 70)

print(f'''
╔══════════════════════════════════════════════════════════════════╗
║                    PREDICTED PODIUM                              ║
╠══════════════════════════════════════════════════════════════════╣
║   1st: {winner['Driver']:<25} ({winner['Team']})
║         Confidence: {winner['Confidence_%']:.1f}%
║
║   2nd: {p2['Driver']:<25} ({p2['Team']})
║         Confidence: {p2['Confidence_%']:.1f}%
║
║   3rd: {p3['Driver']:<25} ({p3['Team']})
║         Confidence: {p3['Confidence_%']:.1f}%
╚══════════════════════════════════════════════════════════════════╝

╔══════════════════════════════════════════════════════════════════╗
║              2025 WORLD CHAMPION PREDICTION                       ║
╠══════════════════════════════════════════════════════════════════╣
║  🏆 {champion['Driver']:<30}
║     Final Points: {champion['Final_Total']:.0f}
║     (Current: {champion['Total_Points']:.0f} + Abu Dhabi: {champion['Predicted_Points']:.0f})
╚══════════════════════════════════════════════════════════════════╝

╔══════════════════════════════════════════════════════════════════╗
║                  MODEL ACCURACY METRICS                           ║
╠══════════════════════════════════════════════════════════════════╣
║  Mean Absolute Error:      {metrics['MAE']:.2f} positions
║  R² Score:                 {metrics['R2']:.3f}
║  Cross-Validation MAE:     {cv_mae:.2f} ± {cv_std:.2f} positions
║  Overall Model Accuracy:   {(1 - cv_mae/10) * 100:.1f}%
║  Prediction Confidence:    ±{cv_mae + cv_std:.1f} positions (95% CI)
╚══════════════════════════════════════════════════════════════════╝
''')

print('=' * 70)
print('Note: Predictions based on 2025 season data using Random Forest ML.')
print('Results may vary due to weather, incidents, strategy, and car setup.')
print('=' * 70)

         ABU DHABI GP 2025 - PREDICTION SUMMARY


PREDICTED PODIUM:
  1st: Max Verstappen (Red Bull Racing Honda RBPT)
  2nd: Oscar Piastri (McLaren Mercedes)
  3rd: George Russell (Mercedes)

PREDICTED CHAMPION: Lando Norris
  Total Points: 391

Model Accuracy (on test data):
  MAE: 2.55 positions
  R2 Score: 0.523

Note: Predictions are based on 2025 season data and ML model.
Actual results may vary due to race incidents, weather, etc.


---

## Methodology and Disclaimer

### Model Architecture
- **Algorithm:** Random Forest Classifier (100 estimators)
- **Features:** Starting Grid, Driver, Team, Track
- **Training Data:** 2025 F1 Season (23 races, 459 entries)
- **Validation:** 5-fold Cross-Validation

### Confidence Calculation
- Ensemble predictions from all decision trees
- 95% Confidence Interval = Mean ± 1.96 × Std Dev
- Individual confidence based on prediction variance

### Limitations
This prediction does not account for:
- Weather conditions (rain, temperature)
- Car upgrades and setup changes
- Race incidents and safety cars
- Tyre degradation and strategy
- Driver form and motivation

**Use this prediction for entertainment and analysis purposes only.**

---
*Created with F1 2025 Season Data Analysis Pipeline*